## Introduction

Modern deep neural networks and the size of training data are becoming extremely large. Training new DNN models on a single GPU node is becoming more and more difficult. Thus, we need to build a large distributed machine learning system, which can use large-scale clusters to train models with big data and achieve equivalent or even better performance than a single node. To realize this goal, we need to optimize the parallelism strategy of machine learning systems for higher speed-up and more efficient training. 

Related work in this area can be divided into the following categories: Model Parallelism, Data Parallelism, and Hybrid Parallelism.

- Data parallelism is the most widely used strategy. It is used for scenarios where the size of training data is large and cannot put into a single machine. To solve this problem, data parallelism allows us to divide the data into multiple shards and distribute them to different nodes. Each node first uses local data which has a small size to train a sub-model, and communicate with other nodes to ensure that the training results from each node can be integrated at certain times, and finally obtain the global model. The parameter update policy (SGD for machine learning/deep learning) for data parallelism can be divided into two categories: asynchronous update and synchronous update. The disadvantages of data parallelism are also obvious. Since each sub-model needs to submit the gradient after each iteration of training, the network communication overhead is very large.

- Model parallelism is used for scenarios where the size of the model is very large and cannot be stored in local memory. In this case, we need to split the model into different modules (e.g., different layers in DNN). Then, each module can be put into different nodes for training. At this time, frequent inter-node communication between different nodes may be required. The performance of model parallelism depends on two aspects, connectivity structure and compute demand of operations. Although model parallelism can solve the problem of large model training, it will also bring us low network traffic and increase training time.

- Hybrid parallelism is the combination of data parallelism and model parallelism.

1. [Data Parallelism](01-Data_Parallelism.ipynb)
1. [Model Parallelism](02-Model_Parallelism.ipynb)
1. [Message Passing](07-Message_Passing.ipynb)
1. [Horovod](08-Horovod.ipynb)
1. [Mixed Precision](06-DDP_Mixed_Precision.ipynb)
1. [Memory Format](05-Memory_Format.ipynb)
1. [Pipeline Parallelism](03-Pipeline_Parallelism.ipynb)
1. [ZeRO](04-ZeRO.ipynb)
1. [PyTorch SLLURM](https://github.com/gfiameni/nvdoc-italy) Working in Progress

# Application process topologies
A Distributed Data Parallel (DDP) application can be executed on multiple nodes where each node can consist of multiple GPU devices. Each node in turn can run multiple copies of the DDP application, each of which processes its models on multiple GPUs.

Let _N_ be the number of nodes on which the application is running and _G_ be the number of GPUs per node. The total number of application
processes running across all the nodes at one time is called the **World Size**, _W_ and the number of processes running on each node
is referred to as the **Local World Size**, _L_.

Each application process is assigned two IDs: a _local_ rank in \[0,_L_-1\] and a _global_ rank in \[0, _W_-1\].

To illustrate the terminology defined above, consider the case where a DDP application is launched on two nodes, each of which has four GPUs. We would then like each process to span two GPUs each. The mapping of processes to nodes is shown in the figure below:

![ProcessMapping](https://user-images.githubusercontent.com/875518/77676984-4c81e400-6f4c-11ea-87d8-f2ff505a99da.png)

While there are quite a few ways to map processes to nodes, a good rule of thumb is to have one process span a single GPU. This enables the DDP application to have as many parallel reader streams as there are GPUs and in practice provides a good balance between I/O and computational costs. In the rest of this tutorial, we assume that the application follows this heuristic.

### Get familiar with the system architecture

In [1]:
! nvidia-smi topo -m

	GPU0	GPU1	GPU2	GPU3	CPU Affinity
GPU0	 X 	NV1	NV1	NV2	0-39
GPU1	NV1	 X 	NV2	NV1	0-39
GPU2	NV1	NV2	 X 	NV1	0-39
GPU3	NV2	NV1	NV1	 X 	0-39

Legend:

  X    = Self
  SYS  = Connection traversing PCIe as well as the SMP interconnect between NUMA nodes (e.g., QPI/UPI)
  NODE = Connection traversing PCIe as well as the interconnect between PCIe Host Bridges within a NUMA node
  PHB  = Connection traversing PCIe as well as a PCIe Host Bridge (typically the CPU)
  PXB  = Connection traversing multiple PCIe switches (without traversing the PCIe Host Bridge)
  PIX  = Connection traversing a single PCIe switch
  NV#  = Connection traversing a bonded set of # NVLinks


### Distributed launch

In [2]:
! python -m torch.distributed.launch

/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
usage: launch.py [-h] [--nnodes NNODES] [--nproc_per_node NPROC_PER_NODE]
                 [--rdzv_backend RDZV_BACKEND] [--rdzv_endpoint RDZV_ENDPOINT]
                 [--rdzv_id RDZV_ID] [--rdzv_conf RDZV_CONF] [--standalone]
                 [--max_restarts MAX_RESTARTS]
                 [--monitor_interval MONITOR_INTERVAL]
                 [--start_method {spawn,fork,forkserver}] [--role ROLE] [-m]
                 [--no_python] [--run_path] [--log_dir LOG_DIR] [-r REDIRECTS]
                 [-t TEE] [--node_rank NODE

### Torchrun

In [3]:
! torchrun

usage: torchrun [-h] [--nnodes NNODES] [--nproc_per_node NPROC_PER_NODE]
                [--rdzv_backend RDZV_BACKEND] [--rdzv_endpoint RDZV_ENDPOINT]
                [--rdzv_id RDZV_ID] [--rdzv_conf RDZV_CONF] [--standalone]
                [--max_restarts MAX_RESTARTS]
                [--monitor_interval MONITOR_INTERVAL]
                [--start_method {spawn,fork,forkserver}] [--role ROLE] [-m]
                [--no_python] [--run_path] [--log_dir LOG_DIR] [-r REDIRECTS]
                [-t TEE] [--node_rank NODE_RANK] [--master_addr MASTER_ADDR]
                [--master_port MASTER_PORT]
                training_script ...
torchrun: error: the following arguments are required: training_script, training_script_args


## Hot to run the examples

In [2]:
#TODO: add reference to NGC container